# Hatree Fock Calculation
#I know that DFT method much better, but I have to use more prooved and sophisticated libraries for MO enegries

**FC = eSC**    # **e**-energy eigenvalues,  **S**-overlaping matrix for orbitals and **C** matric of molecular orbital(row is atomic orbital,columns is MO)
**F = T+V_NE-V_EE**   # Where **T** kinetic energy of electron, **V_NE**-Nuclie attraction, **V_EE**-electron-electron repulsion with (with indistinguishability coeff)


In [10]:
import sys
import math
import numpy as np


In [11]:
# for H2 and for H2O
# alpha values (first col) and contraction coeff (second col)
'''
#----------------------------------------------------------------------
# Basis Set Exchange
# Version v0.9
# https://www.basissetexchange.org
#----------------------------------------------------------------------
#   Basis set: STO-3G
# Description: STO-3G Minimal Basis (3 functions/AO)
#        Role: orbital
#     Version: 1  (Data from Gaussian09)
#----------------------------------------------------------------------


BASIS "ao basis" SPHERICAL PRINT
#BASIS SET: (3s) -> [1s]
H    S
      0.3425250914E+01       0.1543289673E+00
      0.6239137298E+00       0.5353281423E+00
      0.1688554040E+00       0.4446345422E+00
END

Hydrogen is linear combination of three Gaussian (1col) and its coeff(2col) 

'''
#coordinates is a gaussian centers [x, y, z], li-angular momentum, for s --> l = 0

class primitive_gaussian:
    def __init__(self, alpha, coeff, coordinates, l1, l2, l3):
        self.alpha = alpha
        self.coeff = coeff
        self.coordinates = np.array(coordinates)
        self.A = (2.0*alpha/math.pi)**0.75   #+ other terms l1, l2, l3 > 0 (A-normalization constant)
        



In [18]:
# write function for overlaping orbitals molecule = [H1_1s,H2_1s]--> S will be 2x2 matrix 
# because we have two basis 'molecule', for 3-basis mol we can get 3x3 S-matrix
#matrix-vector multiplication with np.dot() or @.
def overlap(molecule):
    
    nbasis=len(molecule) # length = 2
    S = np.zeros([nbasis,nbasis])
    #   ----- loop over each basis functions --------
    # -- loop over S matrix
    for i in range(nbasis):
        for j in range(nbasis):
            nprimitives_basis_i = len(molecule[i]) # extract each gaussian in H1_1s = [H1_pg1a,H1_pg1b ,H1_pg1c] #len = 3 
            nprimitives_basis_j = len(molecule[j]) # extract each gaussian in H2_1s = [H2_pg1a,H2_pg1b ,H2_pg1c] #len = 3  
            #print(f'molecule_i{i} --{nprimitives_basis_i}')
            #print(f'molecule_j{j} --{nprimitives_basis_j}')
            # then we exctract data from (k index) H1_pg1a,H1_pg1b ,H1_pg1c and (l-index)H2_pg1a,H2_pg1b ,H2_pg1c
            for k in range(nprimitives_basis_i):
                for l in range(nprimitives_basis_j):
                    #molecule[i][k].A it like H1_1s calls --> H1_pg1a calls A = (2.0*alpha/math.pi)**0.75 etc
                    #molecule[i][k].A it like H2_1s calls --> H2_pg1a calls A = (2.0*alpha/math.pi)**0.75 etc 
                    N = molecule[i][k].A*molecule[j][l].A # product of normalization const A(1)*A(2)
                    p = molecule[i][k].alpha+molecule[j][l].alpha # sum of STO-3G gaussian 
                    q = molecule[i][k].alpha*molecule[j][l].alpha
                    Q = molecule[i][k].coordinates-molecule[j][l].coordinates # distance between gaussians centers
                    Q2 = np.dot(Q,Q)  # Q*Q dot product
                    #print(p)
                    # S[0,0] , S[0,1] ,S[1,0], S[1,1] fill the matrix by these elements 
                    S[i,j] += N*molecule[i][k].coeff*molecule[j][l].coeff*math.exp(-q*Q2)*(math.pi/p)**(3/2) 
    return S
                    
#-----------------------------------------------------------------------------------------------------------            
            
'''
nbasis=len(molecule)
for i in range(nbasis):
        for j in range(nbasis):
            print(i,'*',j)

0 * 0
0 * 1
1 * 0
1 * 1

S = np.zeros([nbasis,nbasis])
array([[0., 0.],
       [0., 0.]])
'''            

"\nnbasis=len(molecule)\nfor i in range(nbasis):\n        for j in range(nbasis):\n            print(i,'*',j)\n\n0 * 0\n0 * 1\n1 * 0\n1 * 1\n\nS = np.zeros([nbasis,nbasis])\narray([[0., 0.],\n       [0., 0.]])\n"

In [19]:
'''

#----------------------------------------------------------------------
# Basis Set Exchange
# Version v0.9
# https://www.basissetexchange.org
#----------------------------------------------------------------------
#   Basis set: 6-31G
# Description: 6-31G valence double-zeta
#        Role: orbital
#     Version: 1  (Data from Gaussian 09/GAMESS)
#----------------------------------------------------------------------


BASIS "ao basis" SPHERICAL PRINT
#BASIS SET: (4s) -> [2s]
H    S
      0.1873113696E+02       0.3349460434E-01   # 1s
      0.2825394365E+01       0.2347269535E+00
      0.6401216923E+00       0.8137573261E+00
H    S
      0.1612777588E+00       1.0000000   # 2s
END


'''

'\n\n#----------------------------------------------------------------------\n# Basis Set Exchange\n# Version v0.9\n# https://www.basissetexchange.org\n#----------------------------------------------------------------------\n#   Basis set: 6-31G\n# Description: 6-31G valence double-zeta\n#        Role: orbital\n#     Version: 1  (Data from Gaussian 09/GAMESS)\n#----------------------------------------------------------------------\n\n\nBASIS "ao basis" SPHERICAL PRINT\n#BASIS SET: (4s) -> [2s]\nH    S\n      0.1873113696E+02       0.3349460434E-01   # 1s\n      0.2825394365E+01       0.2347269535E+00\n      0.6401216923E+00       0.8137573261E+00\nH    S\n      0.1612777588E+00       1.0000000   # 2s\nEND\n\n\n'

In [30]:
#retrosynthetic analysis starting at end of molecules and working backwords 
#------------------------------------------------------------------------------------------
# STO-3G for 1s orbitals
# 1atom
H1_pg1a = primitive_gaussian(0.3425250914E+01, 0.1543289673E+00, [0,0,0],0,0,0)
H1_pg1b = primitive_gaussian(0.6239137298E+00, 0.5353281423E+00, [0,0,0],0,0,0)
H1_pg1c = primitive_gaussian(0.1688554040E+00, 0.4446345422E+00, [0,0,0],0,0,0)
# 2 atom with displacement 1.2 nm
H2_pg1a = primitive_gaussian(0.3425250914E+01, 0.1543289673E+00, [1.2,0,0],0,0,0)  #1.2 displacment in x direction
H2_pg1b = primitive_gaussian(0.6239137298E+00, 0.5353281423E+00, [1.2,0,0],0,0,0)
H2_pg1c = primitive_gaussian(0.1688554040E+00, 0.4446345422E+00, [1.2,0,0],0,0,0)



H1_1s = [H1_pg1a,H1_pg1b ,H1_pg1c]    # 1s orbital 1-Hydrogen atom consist of 3 primitive gaussianes
H2_1s = [H2_pg1a,H2_pg1b ,H2_pg1c]    # 1s orbital 2-Hydrogen atom 3 primitive gaussianes

molecule = [H1_1s,H2_1s] # molecule with two basis function for 1s orbitals
#nbasis=len(molecule)
print("STO-3G: \n",overlap(molecule))  # S-overlap matrix
print()
print("-------------------------------------------------------------------------")
'''
[[1.         0.67754344]
 [0.67754344 1.        ]]
 
'''
#--------------------------------------------------------------------------------------------
# 6-31G for 1s and 2s orbitals on hydrogen  
# ---- 1atom -----
# for 1s
H1_pg1a = primitive_gaussian(0.1873113696E+02, 0.3349460434E-01, [0,0,0],0,0,0)
H1_pg1b = primitive_gaussian(0.2825394365E+01 , 0.2347269535E+00, [0,0,0],0,0,0)
H1_pg1c = primitive_gaussian(0.6401216923E+00, 0.8137573261E+00, [0,0,0],0,0,0)
# for 2s
H1_pg2a = primitive_gaussian(0.1612777588E+00, 1.0000000, [0,0,0],0,0,0)

#---- 2atom ----
# for 1s
H2_pg1a = primitive_gaussian(0.1873113696E+02, 0.3349460434E-01, [1.2,0,0],0,0,0)
H2_pg1b = primitive_gaussian(0.2825394365E+01 , 0.2347269535E+00, [1.2,0,0],0,0,0)
H2_pg1c = primitive_gaussian(0.6401216923E+00, 0.8137573261E+00, [1.2,0,0],0,0,0)
# for 2s
H2_pg2a = primitive_gaussian(0.1612777588E+00, 1.0000000, [1.2,0,0],0,0,0)

H1_1s = [H1_pg1a,H1_pg1b ,H1_pg1c]    # 1s orbital 1-Hydrogen atom consist of 3 primitive gaussianes
H1_2s=[H1_pg2a] #2s orbital

H2_1s = [H2_pg1a,H2_pg1b ,H2_pg1c]    # 1s orbital 2-Hydrogen atom 3 primitive gaussianes
H2_2s=[H2_pg2a]  #2s orbital
molecule = [H1_1s, H1_2s, H2_1s, H2_2s]
#overlap(molecule) 
print("6-31G: \n",overlap(molecule))
print("-------------------------------------------------------------------------")
'''
ground state in the same place as my incoming
array([[1.        , 0.65829197, 0.38637502, 0.54064104],
       [0.65829197, 1.        , 0.54064104, 0.96323763],
       [0.38637502, 0.54064104, 1.        , 0.65829197],
       [0.54064104, 0.96323763, 0.65829197, 1.        ]])
'''

STO-3G: 
 [[1.         0.67754344]
 [0.67754344 1.        ]]

-------------------------------------------------------------------------
6-31G: 
 [[1.         0.65829197 0.38637502 0.54064104]
 [0.65829197 1.         0.54064104 0.96323763]
 [0.38637502 0.54064104 1.         0.65829197]
 [0.54064104 0.96323763 0.65829197 1.        ]]
-------------------------------------------------------------------------


'\narray([[1.        , 0.65829197, 0.38637502, 0.54064104],\n       [0.65829197, 1.        , 0.54064104, 0.96323763],\n       [0.38637502, 0.54064104, 1.        , 0.65829197],\n       [0.54064104, 0.96323763, 0.65829197, 1.        ]])\n'